In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-03-21 12:10:01.235587: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 12:10:01.235623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [3]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [4]:
from relation_modeling_utils import load_data

train_df = load_data("data/atomic2020_data-feb2021/train.tsv")
dev_df = load_data("data/atomic2020_data-feb2021/dev.tsv")

In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 16

vectorizer = TextVectorization(output_sequence_length=maxlen)
vectorizer.adapt(train_df.text)
vocab_size = len(vectorizer.get_vocabulary())

x_train = vectorizer(train_df.text)
y_train = tf.convert_to_tensor(train_df.label.to_numpy())
x_val = vectorizer(dev_df.text)
y_val = tf.convert_to_tensor(dev_df.label.to_numpy())

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

2022-03-21 12:10:28.537319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 12:10:28.537409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-21 12:10:28.537471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-21 12:10:28.540522: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-03-21 12:10:28.540590: W tensorflow/stream_executor/platform/default/dso_loader

In [6]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val)
)

Epoch 1/20
1685/1685 [==============================] - 19s 11ms/step - loss: 0.7243 - accuracy: 0.6361 - val_loss: 0.8194 - val_accuracy: 0.5295
Epoch 2/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.6422 - accuracy: 0.6494 - val_loss: 0.7989 - val_accuracy: 0.5293
Epoch 3/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.6050 - accuracy: 0.6518 - val_loss: 0.8079 - val_accuracy: 0.5291
Epoch 4/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.5864 - accuracy: 0.6525 - val_loss: 0.8016 - val_accuracy: 0.5287
Epoch 5/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.5762 - accuracy: 0.6535 - val_loss: 0.8144 - val_accuracy: 0.5281
Epoch 6/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.5678 - accuracy: 0.6553 - val_loss: 0.8111 - val_accuracy: 0.5260
Epoch 7/20
1685/1685 [==============================] - 17s 10ms/step - loss: 0.5616 - accuracy: 0.6575 - val_loss: 0.8331 -